In [ ]:
%%writefile productStartUp/functionApp/HttpTrigger1/__init__.py
import azure.functions as func
import os
import tempfile

import json

import numpy as np
import pandas as pd
from operator import itemgetter
import pickle

from io import BytesIO

def find_top_n_indices(data, top=5):
    indexed = enumerate(data)
    sorted_data = sorted(indexed, 
                         key=itemgetter(1), 
                         reverse=True) 
    return [d[0] for d in sorted_data[:top]]

def recommendFromArticle(article_id, top):
    score = []
    for i in range(0, len(ndArtEmbed)):
        if(article_id != i):
            cos_sim = np.dot(ndArtEmbed[article_id], ndArtEmbed[i])/(np.linalg.norm(ndArtEmbed[article_id])*np.linalg.norm(ndArtEmbed[i]))
            score.append(cos_sim)

    _best_scores = find_top_n_indices(score, top)
            
    return _best_scores

def main(req: func.HttpRequest,
        users: func.InputStream,
        embedMatrix: func.InputStream) -> func.HttpResponse:

    global ndArtEmbed

    # Chargement de la dataFrame utilisateurs
    bUsers = users.read()
    dfUsers = pd.read_csv(BytesIO(bUsers), index_col=False)

    # chargement de la matrice d'embedding
    temp_path = tempfile.gettempdir()
    file_name = os.path.join(temp_path, "embedMatrix.pkl")
    with open(file_name, "w+b") as local_file:
        local_file.write(embedMatrix.read())

    with open(local_file.name,'rb') as f:
        ndArtEmbed = pickle.load(f)

    jsonIdUser = req.get_json()

    # récupération de l'id de l'article de référence de l'utilisateur
    articleRef = dfUsers.loc[dfUsers.user_id==jsonIdUser['userId'],['article_ref_id']].values[0,0]
    lReco = recommendFromArticle(articleRef, 5)

    headers = {"Content-Type": "application/json"}
    return func.HttpResponse(
        json.dumps(lReco)
        ,headers=headers
    )


Writing productStartUp/functionApp/HttpTrigger1/__init__.py


# Paramétrage de l'application BookShelf

In [ ]:
%%writefile config.json
{
  "API_URL": "https://bookshelflastversion.azurewebsites.net/api/httptrigger1"
}